###  Prerpcoessing

MAVAN genotypes are previously pruned using plink 1.90. With windows size set at 50bp, sliding step set at 5bp, and VIF threshold set at 2.0. 

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import sys
print (sys.version)

3.5.0 (default, Nov 12 2015, 10:55:08) 
[GCC 4.8.3]


In [4]:
# start pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
conf = SparkConf().setMaster("local").setAppName("App")
sc = SparkContext(conf = conf)
sqlContext=SQLContext(sc)
print(sc)
print(sqlContext)

In [33]:
from operator import add 
from math import log
# PRS calculation on pruned NFP data, 50, 5, VIF=2

#**ATTN: python index starts at 0, so if you want to specify the second column, use 1
#**ATTN: please remove the header of the GWAS file if there is any

# define column number for contents in GWAS

gwas_id=0    # column of SNP ID
gwas_p=7     # column of P value
gwas_or=5    # column of odds ratio
gwas_a1=3    # column of a1 in the GWAS
gwas_maf=4
# defin column number for contents in genfile
geno_id=1  # column number with rsID
geno_start=5 # column number of the 1st genotype
geno_a1 = 3  # column number that contains the reference allele

# List of thresholds:
thresholds=[0.5, 0.3, 0.2, 0.1, 0.05, 0.01, 0.001, 0.0001]

# file delimiters:
GWAS_delim="\t"
GENO_delim=" "
log_or=False
# file names:
home="/home/nyao111/"  #define homefolder path
gwasFiles=home+"PRS_imputed/pgc.cross.noheader.txt"       # Name of GWAS file 



In [37]:
import sys
from PRSfunctions import filterGWASByP, makeGenotypeCheckRef, multiplyOdds, calcPRSFromGeno, getSampleNames, getMaf, labelPRS, writePRS

# Calculat PRS for Emotional Well-being

In [8]:
!echo $SCRATCH

/scratch/vvp-220-aa/


In [9]:
home="/scratch/vvp-220-aa/"
EWBgwas=sc.textFile(home+"GWAS-EWB-noheader-corrected-2.txt")
EWBgwastable=EWBgwas.map(lambda line: line.split("\t"))     
EWBgwastable.count()

22

In [6]:
#== calculate genotype amd fiilter based on EWB GWAS
EWBoddspair=EWBgwastable.map(lambda line: (line[0], float(line[5]))).collectAsMap()
EWBGWAStop=EWBgwastable.map(lambda line: (line[0], line[3])).collectAsMap()

In [11]:
EWBgwasmap=EWBgwastable.map(lambda line: (line[gwas_id], line)).collectAsMap()

In [15]:
import glob, os
import re

fileList={}
for filename in os.listdir(home+"MAVAN_imputed"):
    if filename.endswith(".impute2"):
        chrom=int(re.search("(?<=chr).*(?=.pos)", filename).group(0))
        start=int(re.search("(?<=pos).*(?=-)", filename).group(0))
        #end=int(re.search("(?<=-).*(?=.impu)", filename).group(0))
        fileList[(chrom, start)]=filename


In [13]:

posPair=EWBgwastable.map(lambda line: (line[0],(int(line[1]), int(line[2].strip('"').replace(',', ''))//5000000*5000000+1))).collect()
print (posPair)

[('rs3756290', (5, 130000001)), ('rs2075677', (20, 45000001)), ('rs4958581', (5, 150000001)), ('rs2572431', (8, 10000001)), ('rs193236081', (17, 40000001)), ('rs10960103', (9, 10000001)), ('rs4938021', (11, 110000001)), ('rs139237746', (11, 10000001)), ('rs1557341', (18, 35000001)), ('rs12938775', (17, 1)), ('rs12961969', (18, 35000001)), ('rs35688236', (3, 30000001)), ('rs2150462', (9, 20000001)), ('rs12903563', (15, 75000001)), ('rs7973260', (12, 115000001)), ('rs62100776', (18, 50000001)), ('rs4346787', (6, 25000001)), ('rs4481363', (5, 160000001)), ('rs10838738', (11, 45000001)), ('rs10774909', (12, 115000001)), ('rs6904596', (6, 25000001)), ('rs4481363', (5, 160000001))]


In [16]:
def seekFile(positionList, fileMap):
    result=[]
    try:
        for position in positionList:
            result.append(fileMap[position[1]])
    except:
        print (position[0])
        pass

    return result

inputList=seekFile(posPair, fileList)
inputPaths=set([home+"MAVAN_imputed/"+filename for filename in inputList])
len(inputPaths)

18

In [17]:
EWBgeno=sc.textFile(",".join(inputPaths)).map(lambda line: line.split(' '))
EWBgenoCA=EWBgeno.cache()

In [20]:
EWBgenoFilterByGWASCA=EWBgenoCA.filter(lambda line: line[1] in EWBgwasmap).cache()

In [21]:
EWBgenoFilterByGWASList=EWBgenoFilterByGWASCA.collect()

In [33]:
EWBgenoAmbi=EWBgenoFilterByGWASCA.filter(lambda line: line[3]==bpMap[line[4]]).map(lambda line: (line[1], line[3], line[4])).collect()

In [56]:
EWBgenoAmbi

[('rs10774909', 'C', 'G'),
 ('rs10960103', 'C', 'G'),
 ('rs62100776', 'A', 'T'),
 ('rs2150462', 'G', 'C')]

In [35]:
EWBsnpAmbiSet=set(EWBgenoFilterByGWASCA.filter(lambda line: line[3]==bpMap[line[4]]).map(lambda line: line[1]).collect())

In [50]:
EWBgwasAmbi=EWBgwastable.filter(lambda line: line[0] in EWBsnpAmbiSet).collect()
for line in EWBgwasAmbi:
    print(line)

['rs10960103', '9', '"11,699,270"', 'C', '0.77', '0.0264', '0.0042', '0.024%', '2.14E-10', '"165,380"', 'DS*']
['rs2150462', '9', '"23,316,330"', 'C', '0.74', '-0.0217', '0.0039', '0.018%', '2.66E-08', '"170,907"', '']
['rs62100776', '18', '"50,754,633"', 'A', '0.56', '-0.0252', '0.0044', '0.031%', '8.45E-09', '"105,739"', '"DS**, SWB*"']
['rs10774909', '12', '"117,674,129"', 'C', '0.52', '0.0150', '0.0039', '0.011%', '1.20E-04', '0.0203', '"131,235"']


In [26]:
## calculate allele frequencies for all snps
alleleFreq=EWBgenoFilterByGWASCA.map(lambda line: getMaf(line, rsidCol=geno_id, start=geno_start, a1=geno_a1)).collectAsMap()
#for line in alleleFreq.collect():
   # print(line)

In [38]:
geno_a1=3
EWBgenotype=EWBgenoFilterByGWASCA.map(lambda snp: makeGenotypeCheckRef(snp, freqMap=alleleFreq, gwasMap=EWBgwasmap, idCol=geno_id, a1Col=geno_a1, startCol=geno_start, gwasA1=gwas_a1, gwasMAF=gwas_maf))
count=EWBgenotype.count()
#EWBscore=calPRSFromGeno(EWBgenotype, EWBoddspair, totalcount=count)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 16.0 failed 1 times, most recent failure: Lost task 7.0 in stage 16.0 (TID 599, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/software6/apps/spark/2.0.0/python/lib/pyspark.zip/pyspark/worker.py", line 172, in main
    process()
  File "/software6/apps/spark/2.0.0/python/lib/pyspark.zip/pyspark/worker.py", line 167, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/software6/apps/spark/2.0.0/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/software6/apps/spark/2.0.0/python/pyspark/rdd.py", line 2371, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/software6/apps/spark/2.0.0/python/pyspark/rdd.py", line 2371, in pipeline_fu

In [63]:
csv.register_dialect(
    'outputdialect',
    delimiter = ',',
    lineterminator = '\r\n',
    quoting = csv.QUOTE_NONE)
import csv
def parsePRS(scorelist, sampleSheetPath,idCol=4, dialect=None, skipline=0):
    subjlist=[]
    with open(sampleSheetPath, "r") as csvfile:
        subjs=csv.reader(csvfile, dialect=dialect) 
        for idx, subject in enumerate(subjs):
            if idx>skipline:
                subjlist.append(subject[idCol])
        return zip(subjlist, scorelist)


def writePRS(scores, filePath, dialect=None):
    with open(filePath, "w") as output:
        prsWriter=csv.writer(output, dialect=dialect)
        for score in scores:
            prsWriter.writerow(score)


In [64]:
fullPRS=parsePRS(EWBscore, "SubjectName_imputed_sorted.csv")        
writePRS(fullPRS, "PRS_EWB_corrected_withproxySNPs.csv","outputdialect")

In [31]:

sampleSheet=pd.read_csv("SubjectName_imputed_sorted.csv", header=0, sep=",")
MAVANPRS_EWB=pd.DataFrame(EWBscore)
MAVANPRS_EWB.columns=["PRS"]
MAVANPRS_EWB.index=sampleSheet.iloc[:,1]

In [32]:
MAVANPRS_EWB.to_csv("MAVANPRS_Emotional_well-being.csv")

# Calculate PRS for T2D

In [5]:
gwasfile=sc.textFile(home+"GWAS/DIAGRAMv3.2012DEC17.txt")
prunedsnp=sc.textFile(home+"/MAVAN_prunedSNP.txt")
mavangeno=sc.textFile(home+"MAVAN_imputed/*.impute2")

gwastable=gwasfile.map(lambda line: line.split("\t"))     
gwaspairs=gwasfile.map(lambda line: (line.split("\t")[0], line))

pruneList=prunedsnp.mapPartitions(buildlist).collect()
pruneSet=set(pruneList)
pruneLookup=sc.broadcast(pruneSet)

GWASpruned=gwastable.filter(lambda line: line[0] in pruneLookup.value)  #292232 lines, distinct
GWASprunedsnpID=GWASpruned.map(lambda snp: snp[0]).collect()
GWASprunedsnpset=set(GWASprunedsnpID)

GWASprunedsnpTop=GWASpruned.map(lambda snp: (snp[0], snp[3])).collectAsMap()

genoPruned=mavangeno.filter(lambda line : line.split(' ')[1] in pruneLookup.value).map(lambda x: x.split(" "))
GWASTopStrandBC=sc.broadcast(GWASprunedsnpTop)
bpMap=sc.broadcast({"C":"G", "G":"C", "A":"T", "T":"A"})
MAVANgenotype=genoPruned.map(lambda line:  makeGenotype(line, GWASTopStrandBC.value, bpMap.value))
try:
    test=MAVANgenotype.first()

except:
    print("Calculation Unccessful")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [33]:
print(GWASpruned.map(lambda snp: (snp[0], snp[3])).take(50))

[(u'rs2980319', u'T'), (u'rs3934834', u'C'), (u'rs3766191', u'C'), (u'rs3737728', u'G'), (u'rs9651273', u'G'), (u'rs6671356', u'C'), (u'rs6666280', u'T'), (u'rs4970405', u'G'), (u'rs9659914', u'C'), (u'rs9442373', u'C'), (u'rs2298217', u'T'), (u'rs4970357', u'C'), (u'rs9442380', u'C'), (u'rs4970358', u'A'), (u'rs9442385', u'T'), (u'rs3813200', u'C'), (u'rs2887286', u'T'), (u'rs6603781', u'A'), (u'rs715643', u'C'), (u'rs3818646', u'T'), (u'rs12563338', u'T'), (u'rs4018608', u'C'), (u'rs6668223', u'A'), (u'rs13306651', u'G'), (u'rs2296474', u'A'), (u'rs12403864', u'G'), (u'rs1571150', u'A'), (u'rs3766177', u'C'), (u'rs7531530', u'T'), (u'rs2296716', u'T'), (u'rs9439468', u'G'), (u'rs7540231', u'A'), (u'rs1107910', u'T'), (u'rs6603811', u'C'), (u'rs7545812', u'G'), (u'rs11585349', u'T'), (u'rs12408690', u'G'), (u'rs12141314', u'G'), (u'rs11260616', u'A'), (u'rs6663586', u'A'), (u'rs4648727', u'C'), (u'rs6692959', u'C'), (u'rs6681938', u'C'), (u'rs6664578', u'A'), (u'rs7511905', u'C'), (u'

In [6]:
totalmissing=genoPruned.filter(lambda line: line[1] not in GWASprunedsnpTop).count()

In [9]:
totalSNP=genoPruned.count()

In [7]:
totalmissing ## 7202596 

7202596

In [10]:
totalSNP  #7757777

7757777

In [ ]:
genoPruned100=genoPruned.take(500)
for geno in genoPruned100:
    if geno[1] in GWASprunedsnpTop:
        indicate="Exists"
    else:
        indicate="Missing"
    print(geno[1]+" "+indicate)

In [10]:
home+"GWAS/DIAGRAMv3.2012DEC17.txt"
home+"MAVAN_imputed/*.impute2"
home+"/MAVAN_prunedSNP.txt"

MAVANgenoT2D, T2DGWAS=prepare(home+"GWAS/DIAGRAMv3.2012DEC17-noheader.txt",home+"MAVAN_imputed/*.impute2", home+"/MAVAN_prunedSNP.txt")

[[u'rs2980319', u'1', u'766985', u'T', u'A', u'3.9E-01', u'1.03', u'0.97', u'1.09', u'7575', u'27012'], [u'rs3934834', u'1', u'995669', u'C', u'T', u'4.3E-01', u'1.03', u'0.96', u'1.11', u'4752', u'33536'], [u'rs3766191', u'1', u'1007450', u'C', u'T', u'4.1E-01', u'1.04', u'0.95', u'1.13', u'4752', u'33536'], [u'rs3737728', u'1', u'1011278', u'G', u'A', u'7.3E-01', u'1.01', u'0.96', u'1.06', u'6634', u'49797'], [u'rs9651273', u'1', u'1021403', u'G', u'A', u'6.8E-01', u'1.01', u'0.96', u'1.07', u'4752', u'33536'], [u'rs6671356', u'1', u'1029889', u'C', u'T', u'3.6E-01', u'1.04', u'0.96', u'1.12', u'5426', u'41200'], [u'rs6666280', u'1', u'1036027', u'T', u'C', u'2.8E-01', u'1.04', u'0.97', u'1.12', u'6201', u'48359'], [u'rs4970405', u'1', u'1038818', u'G', u'A', u'3.4E-02', u'1.07', u'1.01', u'1.15', u'8792', u'51411'], [u'rs9659914', u'1', u'1051878', u'C', u'T', u'6.3E-01', u'1.04', u'0.89', u'1.22', u'4752', u'33536'], [u'rs9442373', u'1', u'1052501', u'C', u'A', u'4.9E-01', u'1.02',

In [11]:
MAVANgenoT2D.take(10)

[(u'rs58108140', None),
 (u'rs180734498', None),
 (u'chr1:28590', None),
 (u'rs140337953', None),
 (u'rs116400033', None),
 (u'rs141149254', None),
 (u'rs2462492', None),
 (u'rs143174675', None),
 (u'rs3091274', None),
 (u'rs200769871', None)]

In [14]:
'''pruneCollect=pruneList.collect()
pruneSet=set(pruneCollect)
import pandas as pd
import csv
snpdata=pd.DataFrame(pruneCollect)
snpdata.to_csv(home+"PRS/prunedSNP.txt", \
               header=None, index=None, sep=' ', quoting=csv.QUOTE_NONE, escapechar=' ')'''

In [8]:
pruneLookup=sc.broadcast(pruneSet)

In [71]:
GWASpruned=gwastable.filter(lambda line: line[0] in pruneLookup.value)  #292232 lines, distinct
T2DGWASpruned=T2Dgwastable.filter(lambda line: line[0] in pruneLookup.value) 
T2DGWASpruned.first()

[u'rs2980319',
 u'1',
 u'766985',
 u'T',
 u'A',
 u'3.9E-01',
 u'1.03',
 u'0.97',
 u'1.09',
 u'7575',
 u'27012']

In [12]:
GWASprunedsnpID=GWASpruned.map(lambda snp: snp[0]).collect()
GWASprunedsnpset=set(GWASprunedsnpID)

In [ ]:
T2DGWASprunedsnpID

In [13]:
GWASprunedsnpZpair=GWASpruned.map(lambda snp: (snp[0], float(snp[5]))).collectAsMap()
GWASprunedsnpTopStrand=GWASpruned.map(lambda snp: (snp[0], snp[3])).collectAsMap()

#T2D GWAS ODDspairs and TopStrands


In [14]:
prunedsnpBC=sc.broadcast(GWASprunedsnpset)
mavangeno=sc.textFile(home+"MAVAN_imputed/*.impute2")
genoFilter=mavangeno.filter(lambda line : line.split(' ')[1] in prunedsnpBC.value).map(lambda x: x.split(" "))


In [15]:
genofirst=genoFilter.first()
genofirst[0:10]

[u'---',
 u'rs12562034',
 u'768448',
 u'G',
 u'A',
 u'0.994',
 u'0.006',
 u'0',
 u'0.890',
 u'0.110']

### Calculate Genotype and PRS

In [29]:
GWASprunedsnpZpairBC=sc.broadcast(GWASprunedsnpZpair)
#  Make genotype in 0,1,2,format
MAVANgenotype=genoFilter.map(lambda line: (line[1], makeGenotype(line)))

In [ ]:
## read genotype from 


In [206]:
# Calculate scores first, then filter
MAVANscores=MAVANgenotype.map(lambda snp : (snp[0], snp[1] * GWASprunedsnpZpairBC.value[snp[0]])).cache()
GWASpvaluePair=GWASpruned.map(lambda line: (line[0], float(line[6])))
GWASpvaluePairCache=GWASpvaluePair.cache()
ADDSNP0_0001=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.0001).keys().collect())
ADDSNP0_001=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.001).keys().collect())
ADDSNP0_01=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.01).keys().collect())
ADDSNP0_05=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.05).keys().collect())
ADDSNP0_1=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.1).keys().collect())
ADDSNP0_2=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.2).keys().collect())
ADDSNP0_5=set(GWASpvaluePairCache.filter(lambda line: line[1]<=0.5).keys().collect())

In [30]:
# filter the odds ratio first, apply each to genotype
GWASprunedCA=GWASpruned.cache()
ADDOdds0_0001=filterGWASByP(GWASprunedCA, 0.0001).collectAsMap()
ADDOdds0_001=filterGWASByP(GWASprunedCA, 0.001).collectAsMap()
ADDOdds0_01=filterGWASByP(GWASprunedCA, 0.01).collectAsMap()
ADDOdds0_05=filterGWASByP(GWASprunedCA, 0.05).collectAsMap()
ADDOdds0_1=filterGWASByP(GWASprunedCA, 0.1).collectAsMap()
ADDOdds0_2=filterGWASByP(GWASprunedCA, 0.2).collectAsMap()
ADDOdds0_5=filterGWASByP(GWASprunedCA, 0.5).collectAsMap()

In [51]:
MAVANgenotypeCache=MAVANgenotype.cache()

In [52]:
PRS_ADD0_0001=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_0001)

In [53]:
PRS_ADD0_001=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_001)
PRS_ADD0_01=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_01)
PRS_ADD0_05=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_05)
PRS_ADD0_1=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_1)
PRS_ADD0_2=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_2)
PRS_ADD0_5=calPRSFromGeno(MAVANgenotypeCache, ADDOdds0_5)

In [61]:
MAVANgenotype.saveAsPickleFile("MavanGenotype")

In [56]:
sampleSheet=pd.read_csv("SubjectName_imputed.csv", header=0, sep=",")

scores=np.array([PRS_ADD0_0001,PRS_ADD0_001, PRS_ADD0_01, PRS_ADD0_05, PRS_ADD0_1, PRS_ADD0_2, PRS_ADD0_5])
scores.shape
MAVANPRS_add=pd.DataFrame(scores, index=["P0.0001", "P0.001", "P0.01", "P0.05", "P0.1", "P0.2", "P0.5"])
MAVANPRS_add.columns=sampleSheet.iloc[:,1]

In [60]:
MAVANPRS_add.transpose().to_csv("MAVANPRS_add_imputed.csv")